In [54]:
import pandas as pd
import numpy as np

data = pd.read_csv('apartments.csv')
data.head()

,Price,Bedrooms,Bathrooms,Size
0,ZMW2400000,3.0,2.5,80m²
1,ZMW3250000,3.0,1.0,150m²
2,ZMW5500000,2.0,NaN,1600m²
3,ZMW550000,1.0,1.0,57m²
4,ZMW22100000,NaN,NaN,4000m²


In [55]:
data['Price'] = data['Price'].str.extract('(\d+)').astype(float)
data['Price'] = data['Price'] * 0.03514186

data = data[(data['Price'] >= 5000)]

In [56]:
def size_cleared(value):
    if isinstance(value, (float, int)):
        return float(value)
    elif 'm²' in value:
        return float(value.replace('m²', '').strip())
    elif 'acres' in value:
        acres = float(value.replace('acres', '').strip())
        return acres * 4046.86
    else:
        return np.nan

data['Size'] = data['Size'].apply(size_cleared)
data = data[(data['Size'].isna()) | (data['Size'] >= 15)]

In [57]:
data.describe()

,Price,Bedrooms,Bathrooms,Size
count,1.820000e+02,160.000000,146.000000,7.600000e+01
mean,2.505324e+05,2.743750,2.160959,7.962489e+05
std,5.104530e+05,1.900834,1.490123,5.143291e+06
min,1.581384e+04,1.000000,1.000000,5.700000e+01
25%,4.963788e+04,2.000000,2.000000,5.000000e+02
50%,8.609756e+04,3.000000,2.000000,6.000000e+02
75%,1.932802e+05,3.000000,2.000000,1.618750e+03
max,3.760179e+06,24.000000,16.000000,4.040040e+07


In [58]:
mode_bedrooms = data['Bedrooms'].mode()[0]
data['Bedrooms'] = data['Bedrooms'].fillna(mode_bedrooms)

mode_bathrooms = data['Bathrooms'].mode()[0]
data['Bathrooms'] = data['Bathrooms'].fillna(mode_bathrooms)

In [59]:
data_with_size = data.dropna(subset=['Size'])